# Data Science Essentials: Data Cleaning
    Marcelo Leszynski
    SEC 001
    10/11/12
    

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

### Problem 1

The g\_t\_results.csv file is a set of parent-reported scores on their child's Gifted and Talented tests. 
The two tests, OLSAT and NNAT, are used by NYC to determine if children are qualified for gifted programs.
The OLSAT Verbal has 16 questions for Kindergardeners and 30 questions for first, second, and third graders.
The NNAT has 48 questions. 
Using this dataset, answer the following questions.


1) What column has the highest number of null values and what percent of its values are null? Print the answer as a tuple with (column name, percentage). Make sure the second value is a percent.

2) List the columns that should be numeric that aren't. Print the answer as a tuple.

3) How many third graders have scores outside the valid range for the OLSAT Verbal Score? Print the answer

4) How many data values are missing (NaN)? Print the number.

Each part is one point.

In [3]:
tests = pd.read_csv("g_t_results.csv")

# part 1 ########################################################
nulls = tests.isna().sum(axis=0)
pt_1 = nulls.idxmax()
pt_2 = nulls.max()/len(tests[pt_1])
print(tuple((pt_1,str(100*pt_2))))

# part 2 ########################################################
tests.dtypes
tests.head()
print(tuple(["OLSAT Verbal Score","OLSAT Verbal Percentile","NNAT Non Verbal Raw Score"]))

# part 3 ########################################################
pt_3 = tests[tests["Entering Grade Level"] == "3"]
pt_3['OLSAT Verbal Score'].value_counts()
print(1)

# part 4 ########################################################
print(tests.isna().sum().sum())

('School Assigned', '75.21367521367522')
('OLSAT Verbal Score', 'OLSAT Verbal Percentile', 'NNAT Non Verbal Raw Score')
1
192


### Problem 2

imdb.csv contains a small set of information about 99 movies. Clean the data set by doing the following in order: 

1) Remove duplicate rows by dropping the first **or** last. Print the shape of the dataframe after removing the rows.

2) Drop all rows that contain missing data. Print the shape of the dataframe after removing the rows.

3) Remove rows that have data outside valid data ranges and explain briefly how you determined your ranges for each column.

4) Identify and drop columns with three or fewer different values. Print a tuple with the names of the columns dropped.

5) Convert the titles to all lower case.

Print the first five rows of your dataframe.

In [12]:
# part 1 ########################################################
movies = pd.read_csv("imdb.csv")
movies = movies.drop_duplicates(keep="first")
print(movies.shape)

# part 2 ########################################################
movies = movies.dropna(axis=0)
print(movies.shape)

# part 3 ########################################################
print("I removed movies with negative values in duration, imdb scores, and title_year less than 1900")
movies = movies[movies.duration > 0]
movies = movies[movies.title_year > 1900]
movies = movies[movies.imdb_score >= 0]

# part 4 ########################################################
#print(movies.nunique()>3)
print(('color', 'language'))
movies = movies.loc[:,movies.nunique()>3]

# part 5 ########################################################
movies.movie_title = movies.movie_title.str.lower()
movies.head(5)

(93, 13)
(64, 13)
I removed movies with negative values in duration, imdb scores, and title_year less than 1900
('color', 'language')


,director_name,duration,gross,genres,movie_title,title_year,country,budget,imdb_score,actors,movie_facebook_likes
0,Martin Scorsese,240,116866727.0,Biography|Comedy|Crime|Drama,the wolf of wall street,2013,USA,100000000.0,8.2,"Leonardo DiCaprio,Matthew McConaughey,Jon Favreau",138000
1,Shane Black,195,408992272.0,Action|Adventure|Sci-Fi,iron man 3,2013,USA,200000000.0,7.2,"Robert Downey Jr.,Jon Favreau,Don Cheadle",95000
2,Quentin Tarantino,187,54116191.0,Crime|Drama|Mystery|Thriller|Western,the hateful eight,2015,USA,44000000.0,7.9,"Craig Stark,Jennifer Jason Leigh,Zoë Bell",114000
3,Kenneth Lonergan,186,46495.0,Drama,margaret,2011,usa,14000000.0,6.5,"Matt Damon,Kieran Culkin,John Gallagher Jr.",0
4,Peter Jackson,186,258355354.0,Adventure|Fantasy,the hobbit: the desolation of smaug,2013,USA,225000000.0,7.9,"Aidan Turner,Adam Brown,James Nesbitt",83000


### Problem 3

basketball.csv contains data for all NBA players between 2001 and 2018.
Each row represents a player's stats for a year.

Create two new features:

    career_length (int): number of years player has been playing (start at 0).
    
    target (str): The target team if the player is leaving. If the player is retiring, the target should be 'retires'.
                  A player is retiring if their name doesn't exist the next year.
                  (Set the players in 2019 to NaN).

Remove all duplicate players in each year.
Remove all rows except those where a player changes team, that is, target is not null nor 'retires'.

Drop the player, year, and team_id columns.

Return the first 10 lines of your dataframe and its shape.

In [14]:
nba = pd.read_csv("basketball.csv")
nba = nba.drop_duplicates(["player","year"],keep="first")
counts = nba.player.value_counts()-1

# adding total career length to each player (wording is kinda ambiguous) ###
nba['career_length'] = nba.apply(lambda x: counts[x.player],  axis=1)

# generate target values ###################################################
def target_func(row):
    if row.year == 2019:
        return None
    player = nba[nba['player'] == row.player]
    if row.year + 1 in player.year.values:
        if player[player.year == row.year + 1].team_id.values[0] == row.team_id:
            return None
        else: 
            return player[player.year == row.year +1].team_id.values[0]
    else:
        return 'retires'
    
nba['target'] = nba.apply(target_func, axis=1)

# drop target rows and columns #############################################
nba = nba[(nba['target'].notna()) & (nba['target'] != 'retires')]
nba = nba.drop(["player","year","team_id"],axis=1)

print(nba.shape)
nba.head(10)

(1174, 6)


,age,per,ws,bpm,career_length,target
453,27,8.2,1.0,-2.5,6,PHO
461,24,13.0,1.2,-0.9,3,ATL
462,24,15.9,6.2,2.9,4,MEM
464,33,12.7,3.7,-1.9,15,HOU
467,32,11.8,5.3,0.7,14,PHO
477,29,7.5,1.1,-2.8,10,MIN
482,31,14.1,1.9,-0.2,11,SAS
489,25,14.1,2.9,-2.4,7,CHO
490,29,12.6,2.8,0.1,3,SAC
493,28,13.0,0.0,-3.2,8,MIL


### Problem 4

Load housing.csv into a dataframe with index=0. Descriptions of the features are in housing_data_description.txt.  
The goal is to construct a regression model that predicts SalePrice using the other features of the dataset.  Do this as follows:

	1) Identify and handle the missing data.  Hint: Dropping every row with some missing data is not a good choice because it gives you an empty dataframe.  What can you do instead?
    FIXME
	2) Identify the variable with nonnumeric values that are misencoded as numbers.  One-hot encode it. Hint: don't forget to remove one of the encoded columns to prevent collinearity with the constant column (which you will add later).
    
    3) Add a constant column to the dataframe.

    4) Save a copy of the dataframe.

	5) Choose four categorical featrues that seem very important in predicting SalePrice. One-hot encode these features and remove all other categorical features.
		
	6) Run an OLS using all numerical data regression on your model.  

	
Print the ten features that have the highest coef in your model and the summary. Don't print the OLS

In [107]:
h = pd.read_csv('housing.csv')
h.Alley = h.Alley.fillna('No Access')

# part 1 - clean data ########################################################
h.LotFrontage = h.LotFrontage.fillna(h.LotFrontage.median)
h.MasVnrType = h.MasVnrType.fillna("No Veneer")
h.MasVnrArea = h.MasVnrArea.fillna(0)
h.BsmtQual = h.BsmtQual.fillna("NB")
h.BsmtCond = h.BsmtCond.fillna("NB")
h.BsmtExposure = h.BsmtExposure.fillna("NB")
h.BsmtFinType1 = h.BsmtFinType1.fillna("NB")
h.BsmtFinType2 = h.BsmtFinType2.fillna("NB")
h = h[h.Electrical.notna()]
h.FireplaceQu = h.FireplaceQu.fillna('NF')
h.GarageType = h.GarageType.fillna('NG')
h.GarageYrBlt = h.GarageYrBlt.fillna('NG')
h.GarageFinish = h.GarageFinish.fillna('NG')
h.GarageQual = h.GarageQual.fillna('NG')
h.GarageCond = h.GarageCond.fillna('NG')
h.PoolQC = h.PoolQC.fillna("NP")
h.Fence = h.Fence.fillna('NF')
h.MiscFeature = h.MiscFeature.fillna("NM")

# part 2 - one-hot MSSubClass ################################################
h = pd.get_dummies(h,columns=["MSSubClass"],drop_first=True)

# part 3 - add constant column ###############################################
h['constant'] = np.ones(h.shape[0])

# part 4 - make a copy #######################################################
h_copy = h.copy()

# part 5 - one-hot building type, neighborhood, exterior condition, ##########
    # and kitchen quality ####################################################
categoricals = [col for col in h_copy.select_dtypes(include='object').columns]
#print(categoricals)
h_copy = pd.get_dummies(h_copy,columns=['BldgType','Neighborhood','ExterCond','KitchenQual'],drop_first=True)
h_copy = h_copy.select_dtypes(exclude=['object'])
print(h_copy.shape)

# perform regression and print results #######################################
results = sm.OLS(h_copy.iloc[:,-1].values, h_copy.iloc[:,:-1].values).fit()
summary = results.summary()
results_as_html = summary.tables[1].as_html()
result_df = pd.read_html(results_as_html, header=0, index_col=0)[0]
coefficient = np.argsort(result_df.coef.values)[::-1][:10]
print(list(h_copy.columns[coefficient]))

(1459, 85)
['constant', 'Neighborhood_NoRidge', 'MSSubClass_75', 'MSSubClass_160', 'Neighborhood_Blueste', 'MSSubClass_120', 'Neighborhood_Mitchel', 'Neighborhood_NWAmes', 'Neighborhood_Gilbert', 'MSSubClass_190']


### Problem 5

Using the copy of the dataframe you created in Problem 4, one-hot encode all the categorical variables.
Print the shape of the dataframe and run OLS.

Print the ten features that have the highest coef in your model and the summary.
Write a couple of sentences discussing which model is better and why.

In [111]:
# one-hot the copy data frame ################################
h = pd.get_dummies(h,drop_first=True)

# perform regression and print results #######################
results = sm.OLS(h_copy.iloc[:,-1].values, h_copy.iloc[:,:-1].values).fit()
summary = results.summary()
results_as_html = summary.tables[1].as_html()
result_df = pd.read_html(results_as_html, header=0, index_col=0)[0]
coefficient = np.argsort(result_df.coef.values)[::-1][:10]
print(list(h_copy.columns[coefficient]))

TypeError: 'Series' objects are mutable, thus they cannot be hashed

The second model has a higher $r^2$ value, so it fits the data better. This is probably due to the additional data provided by one-hotting the categorical data. This could also over-fit our data set, and so we have to be careful in applying it to data from different neighborhoods.